#Environment for 3D PIP 🐶 🛺 🚀 🦾


In this notebook there's the environment for a 3 dimensional PIP 🐧 📚

🔖 🦤 *Code based on the standard pybullet [CartPole environment](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/cartpole_bullet.py)*


---

---

* 🎮  **Actions space**: Continous action space, an array of two element indicating the force/acceleration to be applied on x,y (sign and abs value).

* 🎮 **Observations space**: 4 arrays, base linear and angular position and linear and angular velocity. Every array has 2 element,that are the directions of the plane or the interesting Euler angles. Let's make without any nested array, in order to avoid complainments.

  1. ❕  **FORMAT** [Base position: x,y; Euler angles for Base: θ,ϕ; base velocity: ẋ,ẏ; base angular velocity velocity: θ̇,ϕ̇] AKA
  
                    `[x,y, ẋ,ẏ, θ,ϕ, θ̇,ϕ̇̇]`
  I'm ignoring z coordinate and the tilt angle Ψ, cause the cart must stay on the plane, not move in vertical, nore tilt with respect to the plane.

  2. ❗ **CONSTRAINTS**: The PIP can move whitin a plane, so only x and y are limited to the dimensions of the plane itself. The cart has so stay on the plane, so z=0.
   PIP should not tilt with respect to the plane: that's why ψ must be within a certain limit

            *   ϕ, θ, whithin 12 gradi ψ=0;
            *   x, y within 2.4 and 0; z= 0 perchè non vola
Da vedere un attimo con max



ricordati che va installati il pacchetto python pip install GitPython, senno questo non gira!!

In [ ]:
import os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(os.path.dirname(currentdir))
os.sys.path.insert(0, parentdir)

import logging
import math
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import time
import subprocess
import pybullet as p2
import pybullet_data
from pybullet_utils import bullet_client as bc
from pkg_resources import parse_version
import git

logger = logging.getLogger(__name__)


class RealMegaFufiEnv(gym.Env):
  metadata = {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 50}

  def __init__(self, renders=False):
    # set the render
    self._renders = renders
    self._render_height = 200
    self._render_width = 320
    self._physics_client_id = -1

    #set limits for actions and observations
    self.threshold_radians = 12 * 2 * math.pi / 360
    self.threshold_plane = 2.4

    high = np.array([
        self.threshold_plane * 2, self.threshold_plane * 2,
        np.finfo(np.float32).max, np.finfo(np.float32).max,
        self.threshold_radians * 2, self.threshold_radians * 2,
        np.finfo(np.float32).max, np.finfo(np.float32).max,
    ])

    self.observation_space = spaces.Box(-high, high, dtype=np.float32)

    action_lim = np.finfo(np.float32).max
    # force may be applied in the 2 direction of the xy plane
    self.action_space = spaces.Box(low=-action_lim, high=action_lim, shape=(2,), dtype=np.float32)

    self.seed()
    # start the bullet physics server and initialize things up
    self.reset()
    self.viewer = None
    self._configure()

  def _configure(self, display=None):
    self.display = display

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def load_pippa(self, link):
    # Getting git repo for PIP model
    repo_url = 'https://github.com/Gaianeve/Real_Mega_Fufi.git'
    repo_name = 'Real_Mega_Fufi'

    if not os.path.exists(repo_name):
        git.Repo.clone_from(repo_url, repo_name)
        print(f'Repository {repo_name} cloned successfully.')
    else:
        print(f'Repository {repo_name} already exists.')

    # saving PIP urdf directory for the model
    pippa_path = os.path.join(repo_name, 'Robot', 'PIPPA')
    if os.path.exists(pippa_path):
        os.chdir(pippa_path)
        print(f'Changed directory to {pippa_path}.')
    else:
        print(f'Path {pippa_path} does not exist.')
    plane_pos = [0,0,-0.1]
    self.plane = p.loadURDF(os.path.join(pybullet_data.getDataPath(),"plane.urdf", \
                                         plane_pos, flags = flags, useFixedBase= True)
    PIP_position = [0,0,0]
    orientation  = [0,0,0]
    PIP_orientation = p.getQuaternionFromEuler(orientation)
    self.PIPPA_id = p.loadURDF(os.path.join(pippa_path),"PIPPA.urdf",\
                               basePosition = PIP_position,baseOrientation = PIP_orientation,\
                               useFixedBase=False)
    #mettiamo na sferetta nell'origine così possiamo capire dove si trova
    self.sphere = p.loadURDF("sphere_small.urdf", basePosition=[0, 0, 0], globalScaling=1)

    #link del PIP da monitorare per il controllo
    self.link = link

    ## define material for PIP link

    # AL 6082 (Alluminio)
    lateralFriction_Al = 0.3
    restitution_Al = 0.2
    rollingFriction_Al = 0.001
    spinningFriction_Al = 0.01
    linearDamping_Al = 0.04
    angularDamping_Al = 0.1
    contactStiffness_Al = 3e6
    contactDamping_Al = 800

    # AISI 304 (Acciaio Inox)
    lateralFriction_AISI = 0.4
    restitution_AISI = 0.1
    rollingFriction_AISI = 0.001
    spinningFriction_AISI= 0.05
    linearDamping_AISI = 0.03
    angularDamping_AISI = 0.1
    contactStiffness_AISI = 8e6
    contactDamping_AISI = 1500

    # Acciaio Maraging (Maragià per gli amici)
    lateralFriction_Mar = 0.3
    restitution_Mar = 0.2
    rollingFriction_Mar = 0.001
    spinningFriction_Mar = 0.02
    linearDamping_Mar = 0.04
    angularDamping_Mar = 0.15
    contactStiffness_Mar = 3e6
    contactDamping_Mar = 800

    link_indices_Al = [-1, 0, 1, 2, 6, 7, 8, 12, 13, 14]
    link_indices_AISI = [3, 4, 5, 15, 16, 17]
    link_indices_Mar = [9, 10, 11]

    # Al 6082
    for link in link_indices_Al:
      p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_Al,\
                      restitution=restitution_Al, \
                      spinningFriction= spinningFriction_Al, rollingFriction= rollingFriction_Al,\
                      linearDamping=linearDamping_Al, angularDamping=angularDamping_Al,\
                      contactStiffness = contactStiffness_Al,contactDamping = contactDamping_Al)
    # AISI 304
    for link in link_indices_AISI:
      p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_AISI,\
                      restitution=restitution_AISI, \
                      spinningFriction= spinningFriction_AISI, rollingFriction= rollingFriction_AISI,\
                      linearDamping=linearDamping_AISI, angularDamping=angularDamping_AISI,\
                      contactStiffness = contactStiffness_AISI, contactDamping = contactDamping_AISI)
    # Maragià
    for link in link_indices_AISI:
      p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_Mar,\
                      restitution=restitution_Mar, \
                      spinningFriction= spinningFriction_Mar, rollingFriction= rollingFriction_Mar,\
                      linearDamping=linearDamping_Mar, angularDamping=angularDamping_Mar,\
                      contactStiffness = contactStiffness_Mar, contactDamping = contactDamping_Mar)


    return self.PIPPA_id


  def reset(self):
    #    print("-----------reset simulation---------------")
    if self._physics_client_id < 0:
      if self._renders:
        self._p = bc.BulletClient(connection_mode=p2.GUI)
      else:
        self._p = bc.BulletClient()
      self._physics_client_id = self._p._client

      p = self._p
      p.resetSimulation()

      ## loading PIPPA already in the origin
      self.pippa_id = load_pippa(14)
      self.timeStep = 0.02
      p.setGravity(0, 0, -9.8)
      p.setTimeStep(self.timeStep)
      p.setRealTimeSimulation(0)

    p = self._p

    ## Qui forse per lo stato è il caso di prendere la posizione della base e l'inclinazione del link 14, che è l'inclinazione di uno del link del top disk
    # diciamo che per il momento limito la posizione e la velocita lineare della base e l'orientazione e la velocità angolare del disco di sopra e i vincoli li metto lì
    base_position, base_orientation = p.getBasePositionAndOrientation(self.pippa_id)
    base_angles = getEulerFromQuaternion(base_orientation)
    base_linear_velocity, base_angular_velocity = p.getBaseVelocity)(self.pippa_id)

    top_position, top_orientation = p.getLinkState(self.pippa_id, self.link)
    top_angles = getEulerFromQuaternion(top_orientation)
    top_linear_velocity, top_angular_velocity = p.getLinkVelocity(self.pippa_id,self.link)

    self.state = base_position + top_angles + base_linear_velocity + top_angular_velocity
    return np.array(self.state)


  def step(self, action):
    p = self._p
    if self._discrete_actions:
      force = self.force_mag if action == 1 else -self.force_mag
    else:
      force = action[0]

    p.setJointMotorControl2(self.cartpole, 0, p.TORQUE_CONTROL, force=force)
    p.stepSimulation()

    self.state = p.getJointState(self.cartpole, 1)[0:2] + p.getJointState(self.cartpole, 0)[0:2]
    theta, theta_dot, x, x_dot = self.state

    done =  x < -self.x_threshold \
                or x > self.x_threshold \
                or theta < -self.theta_threshold_radians \
                or theta > self.theta_threshold_radians
    done = bool(done)
    reward = 1.0
    #print("state=",self.state)
    return np.array(self.state), reward, done, {}

  def render(self, mode='human', close=False):
    if mode == "human":
      self._renders = True
    if mode != "rgb_array":
      return np.array([])
    base_pos=[0,0,0]
    self._cam_dist = 2
    self._cam_pitch = 0.3
    self._cam_yaw = 0
    if (self._physics_client_id>=0):
      view_matrix = self._p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=base_pos,
        distance=self._cam_dist,
        yaw=self._cam_yaw,
        pitch=self._cam_pitch,
        roll=0,
        upAxisIndex=2)
      proj_matrix = self._p.computeProjectionMatrixFOV(fov=60,
             aspect=float(self._render_width) /
             self._render_height,
             nearVal=0.1,
             farVal=100.0)
      (_, _, px, _, _) = self._p.getCameraImage(
          width=self._render_width,
          height=self._render_height,
          renderer=self._p.ER_BULLET_HARDWARE_OPENGL,
          viewMatrix=view_matrix,
          projectionMatrix=proj_matrix)
    else:
      px = np.array([[[255,255,255,255]]*self._render_width]*self._render_height, dtype=np.uint8)
    rgb_array = np.array(px, dtype=np.uint8)
    rgb_array = np.reshape(np.array(px), (self._render_height, self._render_width, -1))
    rgb_array = rgb_array[:, :, :3]
    return rgb_array

  def configure(self, args):
    pass

  def close(self):
    if self._physics_client_id >= 0:
      self._p.disconnect()
    self._physics_client_id = -1